In [28]:
#[P, P, C, 1B, 2B, 3B, SS, OF, OF, OF]
#maximize average points
using JuMP, Cbc, NamedArrays

raw = readcsv("output.csv");

names = raw[:,1];
salaries = raw[:,2];
position = raw[:, 3];
projectedPoints = raw[:,4];
averagePoints = raw[:,5];
playerTeam = raw[:,6];
opponentTeam = raw[:,7];
n = length(names)

# total salary
salary_cap = 50000

#number of players at each position
no_pitchers = 2
no_catchers = 1
no_first = 1
no_second = 1
no_third = 1
no_ss = 1
no_of = 3
total_players = 10

println(n)
println(playerTeam)
println(opponentTeam)

147
Any[" NYM", " WAS", " PHI", " NYY", " LAD", " SEA", " BAL", " ARI", " ATL", " PIT", " HOU", " LAA", " LAA", " CIN", " NYY", " PHI", " WAS", " ARI", " MIL", " NYY", " SEA", " BAL", " HOU", " NYY", " MIL", " HOU", " CIN", " HOU", " NYY", " ARI", " CIN", " LAA", " WAS", " SEA", " SEA", " ATL", " MIL", " LAD", " MIL", " OAK", " SEA", " ATL", " NYY", " OAK", " MIA", " MIA", " PHI", " ARI", " OAK", " SEA", " NYM", " PHI", " PHI", " PHI", " LAD", " LAD", " PIT", " OAK", " SEA", " NYM", " NYM", " WAS", " HOU", " OAK", " OAK", " LAA", " LAD", " CIN", " CIN", " NYY", " HOU", " NYY", " OAK", " ATL", " PHI", " OAK", " SEA", " SEA", " BAL", " BAL", " NYM", " WAS", " CIN", " NYY", " BAL", " LAA", " LAA", " ATL", " PIT", " CIN", " NYY", " LAA", " ARI", " PIT", " WAS", " MIL", " MIL", " HOU", " OAK", " LAA", " PIT", " MIL", " HOU", " ATL", " MIA", " MIA", " MIA", " LAD", " ARI", " HOU", " LAD", " ARI", " PIT", " WAS", " PIT", " OAK", " NYM", " MIA", " PHI", " LAD", " PIT", " CIN", " LAA", " BAL", 

In [29]:
# could store indexes of all pitchers (or other players) and force (constrain) the sum of binaries at those indexes to equal 2
# or different value for however many players are at that position
#array to store indexes of pitchers, catchers, et c.

pitchers = Int64[]
catchers = Int64[]
first = Int64[]
second = Int64[]
third = Int64[]
shortStop = Int64[]
fielders = Int64[]

for i in 1:n
 if contains(position[i], "P")
  append!(pitchers, i)
 end
 if contains(position[i], "C")
  append!(catchers, i)
 end
 if contains(position[i], "1B")
  append!(first, i)
 end
 if contains(position[i], "2B")
  append!(second, i)
 end
 if contains(position[i], "3B")
  append!(third, i)
 end
 if contains(position[i], "SS")
  append!(shortStop, i)
 end
 if contains(position[i], "OF")
  append!(fielders, i)
 end
end

In [30]:
# NL East
phillies = Int64[]
atlanta = Int64[]
miami = Int64[]
washington = Int64[]

# AL East
yankees = Int64[]
baltimore = Int64[]

# NL Central
milwaukee = Int64[]
pittsburg = Int64[]
# AL Central

# AL West
seattle = Int64[]
angels = Int64[]
oakland = Int64[]

# NL West
dodgers = Int64[]
arizona = Int64[]


for i in 1:n
 if contains(playerTeam[i], " PHI")
  append!(phillies, i)
 end
 if contains(playerTeam[i], " ATL")
  append!(atlanta, i)
 end
 if contains(playerTeam[i], " MIA")
  append!(miami, i)
 end
 if contains(playerTeam[i], " WAS")
  append!(washington, i)
 end
    
 if contains(playerTeam[i], " NYY")
  append!(yankees, i)
 end
 if contains(playerTeam[i], " BAL")
  append!(baltimore, i)
 end
 
 if contains(playerTeam[i], " MIL")
  append!(milwaukee, i)
 end
 if contains(playerTeam[i], " PIT")
  append!(pittsburg, i)
 end

 if contains(playerTeam[i], " SEA")
  append!(seattle, i)
 end
 if contains(playerTeam[i], " LAA")
  append!(angels, i)
 end
 if contains(playerTeam[i], " OAK")
  append!(oakland, i)
 end
    
 if contains(playerTeam[i], " LAD")
  append!(dodgers, i)
 end
 if contains(playerTeam[i], " ARI")
  append!(arizona, i)
 end
end

print(phillies)

[3, 16, 47, 52, 53, 54, 75, 119, 146]

In [33]:
m = Model(solver = CbcSolver())

#binary variable for whether a player is chosen or not
@variable(m, x[1:n], Bin)

#constraints must fit line up, eg. two pitchers, one catcher, one 1B etc
@constraint(m, sum(x[i] for i in pitchers) == no_pitchers)
@constraint(m, sum(x[i] for i in catchers) == no_catchers)
@constraint(m, sum(x[i] for i in first) == no_first)
@constraint(m, sum(x[i] for i in second) == no_second)
@constraint(m, sum(x[i] for i in third) == no_third)
@constraint(m, sum(x[i] for i in shortStop) == no_ss)
@constraint(m, sum(x[i] for i in fielders) == no_of)
@constraint(m, sum(x[i] for i in 1:n) == total_players)

@constraint(m, 5 <= sum(x[i] for i in phillies) <= 6)
@constraint(m, 5 <= sum(x[i] for i in phillies) <= 6)

#total salary must be under 50K
@constraint(m, sum(x .* salaries) <= salary_cap)

#objective to maximize projected points
@objective(m, Max, sum(x .* projectedPoints))

solve(m)

x = getvalue(x)

for i in 1:length(x)
 if x[i] == 1
  println(names[i] , " " , position[i], " ", salaries[i])
 end
end

println(getobjectivevalue(m))

Aaron Nola  P 9900
Zack Godley  P 7600
Rhys Hoskins  OF 5200
Bryce Harper  OF 5100
Gary Sanchez  C 4600
Travis Shaw  3B 4200
Cesar Hernandez  2B 4000
Aaron Altherr  OF 3700
Jesus Aguilar  1B 3200
Pedro Florimon  SS 2400
141.92
